# check

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

# import

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.io import savemat
import scipy
from scipy.linalg import toeplitz
from scipy import signal
import scipy.ndimage as nd
import scipy.ndimage.filters as filters
from ssqueezepy import ssq_stft, issq_stft
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import backend as K
from tensorflow.keras import losses
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Conv2DTranspose, Lambda
from tensorflow.keras.layers import Average, Add, Resizing, concatenate, Activation, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU, ReLU, ELU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras import activations, regularizers
from tensorflow.keras.losses import binary_crossentropy, mse
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16

# functions

In [ ]:
def datagen(img_folder, mask_folder):
    i = 0
    n = os.listdir(img_folder)
    
    while True:
        images = []
        masks  = []
        
        train_msk  = sio.loadmat(mask_folder+'/'+n[i])['temp']
        train_img  = sio.loadmat( img_folder+'/'+n[i])['temp']

        train_msk_T = train_msk.T 
        S_clean = FrST(train_msk_T.ravel())
        train_img_T = train_img.T 
        S_noisy = FrST(train_img_T.ravel())

        #noisy
        images.append(np.dstack(((S_noisy.real - S_noisy.real.min())/(S_noisy.real.max() - S_noisy.real.min()), 
                                 (S_noisy.imag - S_noisy.imag.min())/(S_noisy.imag.max() - S_noisy.imag.min()))))

        #clean
        masks.append(np.dstack((S_clean.real, 
                                S_clean.imag)))

        i+=1
        if(i+1>=len(n)):
            i=0
        yield np.array(images),  np.array(masks)
        
train_gen = datagen('C:/Users/Amir/denoising/newdataset256/train_noisy/train','C:/Users/Amir/denoising/newdataset256/train_clean/train')
val_gen   = datagen('C:/Users/Amir/denoising/newdataset256/val_noisy/val'    ,'C:/Users/Amir/denoising/newdataset256/val_clean/val'    )

# model structure

In [ ]:
def get_model(input_shape):
    
    ##################################### INITIALIZE ##############################################
    
    act     = 'elu'
    act_end = None
    s       = 'same'
    reg     = None
    
    ####################################### ENCODER ###############################################

    inputt = Input(shape=input_shape)
    
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(inputt)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y0= BatchNormalization()(x)
    
    x = Conv2D( 16, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y0)
    x = BatchNormalization()(x)
    x = Conv2D( 16, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y1= BatchNormalization()(x)
    
    x = Conv2D( 32, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y1)
    x = BatchNormalization()(x)
    x = Conv2D( 32, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y2= BatchNormalization()(x)

    x = Conv2D( 64, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y2)
    x = BatchNormalization()(x)
    x = Conv2D( 64, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y3= BatchNormalization()(x)

    ##################################### BOTTLE NECK #############################################

    x = Conv2D(128, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y3)
    x = BatchNormalization()(x)
    
    ####################################### DECODER ###############################################
    
    x = concatenate([y3,x])
    x = Conv2DTranspose( 64, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 64, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)

    x = concatenate([y2,x])
    x = Conv2DTranspose( 32, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 32, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    
    x = concatenate([y1,x])
    x = Conv2DTranspose( 16, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 16, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    
    x = concatenate([y0,x])
    x = Conv2DTranspose(  8, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    
    endd = Conv2D(2, (3, 3), strides=(1,1), activation=act_end, padding=s)(x)

    return Model(inputs=inputt, outputs=endd)

modell = get_model((128,256,2))
modell.summary()

# train

In [ ]:
BATCH_SIZE = 16
n = len(os.listdir('C:/Users/Amir/denoising/newdataset256/train_noisy/train'))//BATCH_SIZE
m = len(os.listdir('C:/Users/Amir/denoising/newdataset256/val_noisy/val'))//BATCH_SIZE
weights_path = 'phase properties'

opt = tf.keras.optimizers.Adam(learning_rate=5e-4)
modell.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

checkpoint     = ModelCheckpoint(weights_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
earlystop      = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks_list = [checkpoint]
history        = modell.fit(train_gen, epochs=100, steps_per_epoch = n, validation_data=val_gen, 
                            validation_steps=m, callbacks=callbacks_list, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')   
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
ll = history.history['loss']
vl = history.history['val_loss']
np.save('ll.npy', ll)
np.save('vl.npy', vl)

# evaluation

In [ ]:
images = []
mytest1= np.zeros((1, 128, 256, 2)).astype('double')

data   = sio.loadmat('C:/Users/Amir/denoising/result/0 dataset/test_noisy/test/ECG_629736.mat')['temp']
cleann = sio.loadmat('C:/Users/Amir/denoising/result/0 dataset/test_clean/test/ECG_629736.mat')['temp']

cleann  = cleann.T
cleann  = cleann.ravel()
S_clean = transform(cleann)

data    = data.T
data    = data.ravel()
S_noisy = transform(data)
S_noisy_mag = np.absolute(S_noisy)
S_noisy_phs = np.angle(S_noisy)

mytest1[0,:,:,:] = np.dstack(((S_noisy.real - S_noisy.real.min())/(S_noisy.real.max() - S_noisy.real.min()), (S_noisy.imag - S_noisy.imag.min())/(S_noisy.imag.max() - S_noisy.imag.min())))
testmodel = load_model('phase properties')
evl             = testmodel.predict(mytest1)
evl = np.array(evl)
xx = evl[0,:,:,0] + 1j * evl[0,:,:,1]

f, axarr = plt.subplots(3,2, figsize=(50,30))

axarr[0,0].plot(data)
axarr[0,1].plot(data)
ecg = itransform(xx)
axarr[1,0].plot(ecg)
axarr[1,1].plot(ecg)
axarr[2,0].plot(cleann)
axarr[2,1].plot(cleann)
axarr[2,1].plot(ecg)